In [1]:
pip install -q -U text2text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.2/71.2 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 17.4 MB/s eta 0:00:00


In [2]:
import requests

def get_data(lang_code="en"):
  url = f"https://raw.githubusercontent.com/deepmind/xquad/master/xquad.{lang_code}.json"
  r = requests.get(url)
  d = r.json()
  corpus = {}
  i = 0
  for a in d["data"]:
    for p in a["paragraphs"]:
      c = p["context"]
      for qa in p["qas"]:
        q = qa["question"]
        corpus[i] = f"{q} {c}".lower()
        i += 1
  return corpus

In [3]:
import text2text as t2t
#lang_codes = ["en","es","de","el","ru","tr","ar","vi","th","zh","hi","ro"]
corpus = get_data(lang_code="en")
index = t2t.Handler(list(corpus.values())).index(ids=list(corpus.keys()))

Better speed can be achieved with apex installed.


Downloading:   0%|          | 0.00/271 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/909 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Creating index with 128104 dimensions.


In [4]:
query = "who did tesla partner with?"
dist, pred_ids = index.search([query.lower()], k=1)
corpus[pred_ids[0][0]]

"who did tesla partner with in 1886? after leaving edison's company tesla partnered with two businessmen in 1886, robert lane and benjamin vail, who agreed to finance an electric lighting company in tesla's name, tesla electric light & manufacturing. the company installed electrical arc light based illumination systems designed by tesla and also had designs for dynamo electric machine commutators, the first patents issued to tesla in the us."

In [5]:
t2t.Handler(["[CONTEXT] who did tesla partner with?"]).respond()

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

["Apple, that's a pretty nice company if you ask me."]

In [6]:
t2t.Handler(["[CONTEXT] who did tesla partner with?"]).respond(knowledge_base=(corpus,index))

['Robert Lane and Benjamin Vail.']

In [13]:
history = []
while True:
  user_input = input('User: ')
  history.append(user_input)
  res = t2t.Handler([f"[CONTEXT] {' EOS '.join(history[-5:])}"]).respond(knowledge_base=(corpus,index))[0]
  history.append(res)
  print(f"T2T: {res}")

User: What is the difference between special relativity and general relativity?
T2T: Special relativity is a theory of relativity. It describes the properties of a curved object, called a sphere.
User: Then what about general relativity?
T2T: This theory best explains gravity. The object moving freely in a gravitational field travels under their own inertia in straight lines through curved space-time.
User: What is the curvature of space and time?
T2T: It defines the shortest space-time path between two space-time events.
User: What is the NFL about?
T2T: The NFL is an American football league founded in 1922.


KeyboardInterrupt: ignored